## 0. Prerequisites

### 0.1. Install and initialize libraries and constants

#### Download Training Outputs
Google colabs has no persistant memory, thus it needs to be stored and retrieved on Google Drive

In [ ]:
! gdown 16vZmWPgR2wPPG_nvVwlxhwUIac-OgQeW
!unzip Training_results.zip

#### Prepare Environment
I'll be using Keras/Tensorflow for the age-classification, and OpenCV for webcam data-handling and face identification

In [ ]:
from base64 import b64decode, b64encode
from google.colab.output import eval_js

from IPython.display import display, Javascript
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
import os
from tensorflow.keras.models import load_model
import io
import numpy as np
from IPython.display import clear_output
from time import sleep
IMG_SHAPE = [640, 480]
IMG_QUALITY = 0.8

model = load_model("/content/Training_results/MPAA_cnn_model_checkpoint.keras") # My age-classification model, see CS Capstone Training.ipynb for more detail
age_ranges = ['PG', 'PG13', 'R']

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml') #helper model to locate faces to feed into my model.

### 0.2. JavaScript workaround to grab live webcam stream
Google colabs is a cloud based environment and can't see any of the attached physical devices. As such, a JS workaround to capture the webcam is necessary.

In [ ]:
def start_stream():
    js = Javascript(f'''
    const IMG_SHAPE = {IMG_SHAPE};
    const IMG_QUALITY = {IMG_QUALITY};
    ''' + '''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
        stream.getVideoTracks()[0].stop();
        video.remove();
        div.remove();
        video = null;
        div = null;
        stream = null;
        imgElement = null;
        captureCanvas = null;
        labelElement = null;
    }

    function onAnimationFrame() {
        if (!shutdown) {
            window.requestAnimationFrame(onAnimationFrame);
        }
        if (pendingResolve) {
            var result = "";
            if (!shutdown) {
                captureCanvas.getContext('2d').drawImage(video, 0, 0, IMG_SHAPE[0], IMG_SHAPE[1]);
                result = captureCanvas.toDataURL('image/jpeg', IMG_QUALITY)
            }
            var lp = pendingResolve;
            pendingResolve = null;
            lp(result);
        }
    }

    async function createDom() {
        if (div !== null) {
            return stream;
        }

        div = document.createElement('div');
        div.style.border = '2px solid black';
        div.style.padding = '3px';
        div.style.width = '100%';
        div.style.maxWidth = '600px';
        document.body.appendChild(div);

        const modelOut = document.createElement('div');
        modelOut.innerHTML = "<span>Status: </span>";
        labelElement = document.createElement('span');
        labelElement.innerText = 'No data';
        labelElement.style.fontWeight = 'bold';
        modelOut.appendChild(labelElement);
        div.appendChild(modelOut);

        video = document.createElement('video');
        video.style.display = 'block';
        video.width = div.clientWidth - 6;
        video.setAttribute('playsinline', '');
        video.onclick = () => { shutdown = true; };
        stream = await navigator.mediaDevices.getUserMedia(
            {video: { facingMode: "environment"}});
        div.appendChild(video);

        imgElement = document.createElement('img');
        imgElement.style.position = 'absolute';
        imgElement.style.zIndex = 1;
        imgElement.onclick = () => { shutdown = true; };
        div.appendChild(imgElement);

        const instruction = document.createElement('div');
        instruction.innerHTML =
            '<span style="color: red; font-weight: bold;">' +
            'When finished, click here or on the video to stop this demo</span>';
        div.appendChild(instruction);
        instruction.onclick = () => { shutdown = true; };

        video.srcObject = stream;
        await video.play();

        captureCanvas = document.createElement('canvas');
        captureCanvas.width = IMG_SHAPE[0]; //video.videoWidth;
        captureCanvas.height = IMG_SHAPE[1]; //video.videoHeight;
        window.requestAnimationFrame(onAnimationFrame);

        return stream;
    }
    async function takePhoto(label, imgData) {
        if (shutdown) {
            removeDom();
            shutdown = false;
            return '';
        }

        var preCreate = Date.now();
        stream = await createDom();

        var preShow = Date.now();
        if (label != "") {
            labelElement.innerHTML = label;
        }

        if (imgData != "") {
            var videoRect = video.getClientRects()[0];
            imgElement.style.top = videoRect.top + "px";
            imgElement.style.left = videoRect.left + "px";
            imgElement.style.width = videoRect.width + "px";
            imgElement.style.height = videoRect.height + "px";
            imgElement.src = imgData;
        }

        var preCapture = Date.now();
        var result = await new Promise((resolve, reject) => pendingResolve = resolve);
        shutdown = false;

        return {
            'create': preShow - preCreate,
            'show': preCapture - preShow,
            'capture': Date.now() - preCapture,
            'img': result,
        };
    }
    ''')
    display(js)

def take_photo(label, img_data):
    data = eval_js(f'takePhoto("{label}", "{img_data}")')
    return data

### 0.3. Define Python functions to interpret webcam stream, classify detected faces, and create the prediction overlay
OpenCV reduces the framerate considerably, as does the Keras age-classification. To reduce the annoyingly choppy framerate I've created a transparent overlay for the prediction results. The transparent overlay improves the performance a bit, but is still noticably choppy compared to the webcam stream.

In [ ]:
def js_response_to_image(js_response) :
    _, b64_str = js_response['img'].split(',')
    jpeg_bytes = b64decode(b64_str)
    im_arr = np.frombuffer(jpeg_bytes, dtype=np.uint8)  # im_arr is one-dim Numpy array
    img = cv2.imdecode(im_arr, flags=cv2.IMREAD_COLOR)

    #image = np.array(jpeg_bytes,dtype="uint8")
    return img
# Defining a function to shrink the detected face region by a scale for better prediction in the model.

def shrink_face_roi(x, y, w, h, scale=0.9):
    wh_multiplier = (1-scale)/2
    x_new = int(x + (w * wh_multiplier))
    y_new = int(y + (h * wh_multiplier))
    w_new = int(w * scale)
    h_new = int(h * scale)
    return (x_new, y_new, w_new, h_new)
# Defining a function to create the predicted age overlay on the image by centering the text.

def create_age_text(img, text, pct_text, x, y, w, h):

    # Defining font, scales and thickness.
    fontFace = cv2.FONT_HERSHEY_SIMPLEX
    text_scale = 1.2
    yrsold_scale = 0.7
    pct_text_scale = 0.65

    # Getting width, height and baseline of age text and "years old".
    (text_width, text_height), text_bsln = cv2.getTextSize(text, fontFace=fontFace, fontScale=text_scale, thickness=2)
    (yrsold_width, yrsold_height), yrsold_bsln = cv2.getTextSize("years old", fontFace=fontFace, fontScale=yrsold_scale, thickness=1)
    (pct_text_width, pct_text_height), pct_text_bsln = cv2.getTextSize(pct_text, fontFace=fontFace, fontScale=pct_text_scale, thickness=1)

    # Calculating center point coordinates of text background rectangle.
    x_center = x + (w/2)
    y_text_center = y + h + 20
    y_yrsold_center = y + h + 48
    y_pct_text_center = y + h + 75

    # Calculating bottom left corner coordinates of text based on text size and center point of background rectangle calculated above.
    x_text_org = int(round(x_center - (text_width / 2)))
    y_text_org = int(round(y_text_center + (text_height / 2)))
    x_yrsold_org = int(round(x_center - (yrsold_width / 2)))
    y_yrsold_org = int(round(y_yrsold_center + (yrsold_height / 2)))
    x_pct_text_org = int(round(x_center - (pct_text_width / 2)))
    y_pct_text_org = int(round(y_pct_text_center + (pct_text_height / 2)))

    face_age_background = cv2.rectangle(img, (x-1, y+h), (x+w+1, y+h+94), (0, 100, 0, 255), cv2.FILLED)
    face_age_text = cv2.putText(img, text, org=(x_text_org, y_text_org), fontFace=fontFace, fontScale=text_scale, thickness=2, color=(255, 255, 255,255), lineType=cv2.LINE_AA)
    yrsold_text = cv2.putText(img, "Max Rating", org=(x_yrsold_org, y_yrsold_org), fontFace=fontFace, fontScale=yrsold_scale, thickness=1, color=(255, 255, 255,255), lineType=cv2.LINE_AA)
    pct_age_text = cv2.putText(img, pct_text, org=(x_pct_text_org, y_pct_text_org), fontFace=fontFace, fontScale=pct_text_scale, thickness=1, color=(255, 255, 255,255), lineType=cv2.LINE_AA)

    return (face_age_background, face_age_text, yrsold_text)
    # Defining a function to find faces in an image and then classify each found face into age-ranges defined above.

def classify_age(img):


    blank_image = np.zeros((480,640,4), np.uint8)
    # Detecting faces in the image using the face_cascade loaded above and storing their coordinates into a list.
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.2, minNeighbors=6, minSize=(100, 100))
    #print("{faces} faces found.")

    # Looping through each face found in the image.
    for i, (x, y, w, h) in enumerate(faces):

        # Drawing a rectangle around the found face.
        face_rect = cv2.rectangle(blank_image, (x, y), (x+w, y+h), (0, 100, 0, 255), thickness=2)

        # Predicting the age of the found face using the model loaded above.
        x2, y2, w2, h2 = shrink_face_roi(x, y, w, h)
        face_roi = img[y2:y2+h2, x2:x2+w2]
        face_roi = cv2.resize(face_roi, (200, 200))
        face_roi = face_roi.reshape(-1, 200, 200, 3)
        face_age = age_ranges[np.argmax(model.predict(face_roi, verbose=0))]
        face_age_pct = f"({round(np.max(model.predict(face_roi, verbose=0))*100, 2)}%)"

        # Calling the above defined function to create the predicted age overlay on the image.
        face_age_background, face_age_text, yrsold_text = create_age_text(blank_image, face_age, face_age_pct, x, y, w, h)

        # print(f"Age prediction for face {i+1} : {face_age} years old")

    return blank_image

## 1. Perform real-time object detection in webcam stream

In [ ]:
start_stream()
img_data = ''
while True:
    js_response = take_photo('Capturing...Overlay may be choppy', img_data)
    if not js_response:
        break
    captured_img = js_response_to_image(js_response)
    result = classify_age(captured_img)
    img_1 = cv2.imencode('.png', result)
    buf = io.BytesIO(img_1[1])
    img_64 = b64encode(buf.getvalue())
    img_str = img_64.decode('utf-8')
    img_data = 'data:image/png;base64,' + img_str